This notebook generates the C++ code used to calculate the steady-state response of the TF-chromatin model away from thermodynamic equilibrium (Fig. 2D) using the MTT to calculate the coefficients of the input-output function.

It uses code that was used in previous work to calculate GRF sharpness. This is not strictly necessary here, and a much simpler code could be used instead. However that was well tested for accuracy and so this should apply here as well. Also, it is used for determining whether the input-output function is monotonic or not for the calculations in Fig.S3.

It uses Eigen (tried with version 3.3.7 or 3.40), boost (tried with version 1.76.0, 1.81.0) and code from the following repositories:

writescripts.py file and the files it depends on, in GeneRegulatoryFunctions repository: https://github.com/rosamc/GeneRegulatoryFunctions.git

(doublechecked at the end with commit bdc925406b59f266ae0918932403834289183db9)

Code to solve polynomials with high accuracy: https://github.com/kmnam/polynomials.git (commit af5a8318a6d637680033858a9b902c6d02eff613)




In [1]:
import numpy as np
import sys, os,re

path_to_eigen="/Users/rosamartinezcorral/Documents/eigen-3.4.0"
path_to_eigen="/Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731"
path_to_polynomialsmultipr="./repos_commit/polynomials/include/polynomial/" # using multiprecision types for root finding

path_to_utilsGRF="./repos_commit/GeneRegulatoryFunctions/utilsGRF/" #GeneRegulatoryFunctions repo
path_to_utilsGRFpy=path_to_utilsGRF
sys.path.append(path_to_utilsGRFpy)
import writescripts 

ModuleNotFoundError: No module named 'writescripts'

In [2]:
edges0=[(1,'ax_0_0-x',2),(2,'bx_x_0',1),
      (1,'aP_0_0',3),(3,'bP_P_0',1),
      (2,'aP_x_0',4),(4,'bP_xP_0',2),
      (3,'ax_P_0-x',4),(4,'bx_xP_0',3),
      ]
edges1=[(1,'kopen0',5),(5,'kclose0',1),
       (2,'kopenx',6),(6,'kclosex',2),
       (3,'kopenP',7),(7,'kcloseP',3),
       (4,'kopenxP',8),(8,'kclosexP',4)]
alledges=[]
n=4
conf=re.compile(r"_0(-)|_0$")
for edge in edges0:
    alledges.append(edge)
    n0,lab,n1=edge
    n0=n0+n
    n1=n1+n
    lab=conf.sub(r'_1\1',lab)
    alledges.append((n0,lab,n1))
alledges.extend(edges1)
print("Graph edges are:")
print(alledges)
MTTfolder=path_to_utilsGRFpy
basename='graph'

parlist=[x[1] for x in alledges]
for pnum,par in enumerate(parlist):
    if '-x' in par:
        parlist[pnum]=parlist[pnum].replace('-x','')
parlist.extend(["q3","q4","q7","q8"])
print("Parameters are")
print(parlist)

Graph edges are:
[(1, 'ax_0_0-x', 2), (5, 'ax_0_1-x', 6), (2, 'bx_x_0', 1), (6, 'bx_x_1', 5), (1, 'aP_0_0', 3), (5, 'aP_0_1', 7), (3, 'bP_P_0', 1), (7, 'bP_P_1', 5), (2, 'aP_x_0', 4), (6, 'aP_x_1', 8), (4, 'bP_xP_0', 2), (8, 'bP_xP_1', 6), (3, 'ax_P_0-x', 4), (7, 'ax_P_1-x', 8), (4, 'bx_xP_0', 3), (8, 'bx_xP_1', 7), (1, 'kopen0', 5), (5, 'kclose0', 1), (2, 'kopenx', 6), (6, 'kclosex', 2), (3, 'kopenP', 7), (7, 'kcloseP', 3), (4, 'kopenxP', 8), (8, 'kclosexP', 4)]
Parameters are
['ax_0_0', 'ax_0_1', 'bx_x_0', 'bx_x_1', 'aP_0_0', 'aP_0_1', 'bP_P_0', 'bP_P_1', 'aP_x_0', 'aP_x_1', 'bP_xP_0', 'bP_xP_1', 'ax_P_0', 'ax_P_1', 'bx_xP_0', 'bx_xP_1', 'kopen0', 'kclose0', 'kopenx', 'kclosex', 'kopenP', 'kcloseP', 'kopenxP', 'kclosexP', 'q3', 'q4', 'q7', 'q8']


In [3]:
obj=writescripts.PrepareFilesNoneq(edgelist=alledges,varGRF='x',concvars=['x'],parlist=parlist,MTTfolder=MTTfolder,graphbasename=basename)
obj.write_execute_parse()

obj.simpify_rhos()

removing  ./repos_commit/GeneRegulatoryFunctions/utilsGRF/graph-parsed.txt
removing  ./repos_commit/GeneRegulatoryFunctions/utilsGRF/graph-6.txt
removing  ./repos_commit/GeneRegulatoryFunctions/utilsGRF/graph-7.txt
removing  ./repos_commit/GeneRegulatoryFunctions/utilsGRF/graph-5.txt
removing  ./repos_commit/GeneRegulatoryFunctions/utilsGRF/graph-4.txt
removing  ./repos_commit/GeneRegulatoryFunctions/utilsGRF/graph-1.txt
removing  ./repos_commit/GeneRegulatoryFunctions/utilsGRF/graph-3.txt
removing  ./repos_commit/GeneRegulatoryFunctions/utilsGRF/graph-2.txt
removing  ./repos_commit/GeneRegulatoryFunctions/utilsGRF/graph-8.txt
removing  ./repos_commit/GeneRegulatoryFunctions/utilsGRF/graph.txt
executing MTT
0


In [4]:
num="q3*(rho3)+q4*(rho4)+q7*(rho7)+q8*(rho8)"
den="1*(rho1+rho2+rho3+rho4+rho5+rho6+rho7+rho8)"

obj.parse_GRF(num,den)

0
1


In [3]:
fname='./bin/twoconf_x_P_higherac_3.cpp'
funcname='GRF_xP'
#obj.write_pybind_interface(fname=fname, funcname=funcname)


Then open the .cpp file and manually edit it so that the type of the parameters in GRF_xP_x and rhos_GRF_xP_x is fixed to long double (instead of T). Using high precision types is not feasible due to memory issues when running the code. 

In [4]:
#fname="./bin/twoconf_x_P_higherac.cpp" #note that here the python parameters are treated as long double types, and then everything else is done with whatever T precision. Otherwise for some cases it produces segmentation fault.
file=fname
#objectnamemac=fname.replace(".cpp","")
objectnamemac=fname.replace(".cpp","")
print(objectnamemac)

compilestringmac="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I  %s -I %s -lmpfr -lmpc -I %s  `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(path_to_eigen, path_to_polynomialsmultipr,path_to_utilsGRF,file,objectnamemac)
#compilestringmac="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I %s -I %s -I /opt/homebrew/include -I /Users/rosamartinezcorral/Documents/eigen-3.4.0 -L /opt/homebrew/lib -lmpfr -lmpc `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(path_to_polynomialsmultipr,path_to_utilsGRF,file,objectnamemac)


!$compilestringmac

./bin/twoconf_x_P_higherac_3
clang: error: no such file or directory: './bin/twoconf_x_P_higherac_3.cpp'
